# RRT for Complex Systems
# Carson Forsyth

# Goal
This tutorial will walk you through how to create an integrated OMPL and PyBullet program modeling constrianed systems.  OMPL will be used for planning and PyBullet will be used to render and create the simulation.  The scenario we will be using is getting a snake robot to plan and then execute a path through a maze from one point to another.

# Motivation
The motivation for this project is versatility of snake robots and the beneficial aspects that they come with when used in certain situations.  Snake robots work well in space, cluttered ground, and narrow environments.  They work well in space because they have a small size and can easily float down the side of a hallway instead of taking up a large amount of space otherwise.  They also work well in clusttered and rugged terrain because they are more flexible and less prone to getting stuck or tipping over like some other robots.  Due to snake robots' small size, they are also much more capable of fitting into areas that are smaller.  They can navigate down tubes or in cracks in certain areas.

# Related Work
Even though I chose to use Rapidly Expanding Random Trees, there is no short of different algorithms that can be used to plan for different environments.  Besides RRT and its many variants such as RRT*, there also exists Probabilistic Road Map (PRM), Voronoi Diagrams, A*, D*, and even evolutionary algorithms.  Testing out how each of these performs in different situations is a good approach to a research topic.  The paper, Survey of Robot 3D Path Planning Algorithms, the authors analysis a large number of different approaches to robot path planning and find where some work and others do not.  This paper is not only useful to compare different planning algorithms, but also to learn about new algorithms that you may not have heard about yet.
P. Petkov, L. Yang, J. Qi, D. Song, J. Xiao, J. Han, and Y. Xia, "Survey of Robot 3D Path Planning Algorithms", Journal of Control Science and Engineering, vol. 2016, 2016, https://doi.org/10.1155/2016/7426913

The Rapidly Expanding Random Tree algorithm was developed in 1998 and has grown in large popularity since then, especially in the robotics field.  This is the algorithm that I chose to utilize in my project.  The original paper for it describes the inner workings of it and propositions as to how it can be used.  They discuss some of the beneficial properties of using this algorithm such as it being biased towards exploring previously unexplored regions and that it is probabilistically complete.  This means that as time approaches infinity and the number of branches increases, the gurantee that a solution will be found if one exists increases.
LaValle, Steven M. (October 1998). "Rapidly-exploring random trees: A new tool for path planning" (PDF). Technical Report. Computer Science Department, Iowa State University (TR 98–11).
 
In Yang et. al, the researchers are planning for a snake robot in an unknown environment, but they do not have an all knowing view as in my experiment.  Their approach is similar to how I implemented mine in that they are generating waypoints that the robot is following.  We both chose to conduct this in a similar way, by using the angle heading of the robot to aim towards the goal.  They are also planning in a maze like environment.  The researchers also used RRT and a variant ERRT to test out planning.
W. Yang, G. Wang, and Y. Shen, "Perception-Aware Path Finding and Following of Snake Robot in Unknown Environment", International Conference on Intelligent Robots and System, 2020

"Study on Snake Robot Design for Investigating Rough Terrain" is another research paper that discusses how snake are able to move through the motion that their bodies make.  They also mention a snake type robot that is being developed by NASA to help explore other planets.  This NASA snake robot is able to more safely explore the terrain since it isn't as susceptable to tipping over as normal rovers are.  Additionally, the expected price of these robots are much cheaper compared to the rovers.
V. Tran and S. Shin, "Study on Snake Robot Design for Investigating Rough Terrain", Kumoh National Institute of Technolog, 2017.

"A review on modelling, implementation, and control of snake robots" discusses previous examples of how snake robots have been modeled and implemented in the past.  In addition, they also present a discussion of different control mechanisms for the implementations.  The paper discuss movement for snake robots that move in 2D, as well as ones in 3D.
P. Liljebäck, K.Y. Pettersen, Ø. Stavdahl, J.T. Gravdahl, A review on modelling, implementation, and control of snake robots, Robotics and Autonomous Systems, Volume 60, Issue 1, 2012, Pages 29-40, ISSN 0921-8890, https://doi.org/10.1016/j.robot.2011.08.010.

# Approach
There are two main technologies that were used in this project, these being the Open Motion Planning Library (OMPL) and PyBullet.  I used OMPL to model, constrain, and plan the path for the snake robot from start to goal.  OMPL used Rapidly Expanding Random Trees (RRT) to search the state space for valid states.  A provided data file would be read in with data about where the obstacles were to avoid as well as other configuration constants.  Once the solution path had been found, the path was sampled to create a set of waypoints that would then be passed to the simulation and snake robot.
The snake robot was simulated using PyBullet, a physics library ported to Python.  PyBullet has a lot of support for function that deal with collision and simulation in three dimensions.  PyBullet then generated a visual representation of the snake and the maze environment.  Additionally, the waypoints generated from OMPL were read in and displayed as well as visual, but collisionless, nodes.  A controller was also defined that would guide the snake between waypoints and this was used in the running simulation.  Finally, the simulation was displayed and run and you can watch as the snake travels towards its goal.  Once reaching its goal, it would stop, as it had succeeded.

# Planning with OMPL
First we will demonstrate how to create a plan using OMPL.  We will start with a very simple scenarion and then slowly increase the complexity.  OMPL is the Open Motion Planning Library, it is originally written in C++, but you can download the python bindings for it to be able to program in Python with it.

# Simple planning
First we will plan in an environment with a point robot and no obstacles.  The only constraints will be the area the agent can be.  In this case, a 2 x 2 area.
![title](images/simple.png)

In [5]:
# all the imports we'll need for this OMPL tutorial
from math import sin, cos
from functools import partial
import sys
from os.path import abspath, dirname, join
import sys
# py bindings should be in the same directory
sys.path.insert(0, join(abspath(''), 'py-bindings'))
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og
import data
import math
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [6]:
# define the function to check if a state is valid
def isStateValid(spaceInformation, state):
    return spaceInformation.satisfiesBounds(state)

In [24]:
TOLERANCE = 0.1
START_LOC = (-0.8, -0.8)
GOAL_LOC = (0.8, 0.8)

# main function to plan path
def plan():
    # construct the SE2 state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    # set highest and lowest values for each vector component
    bounds.setLow(-1)
    bounds.setHigh(1)
    space.setBounds(bounds)

    # handle setup for planning
    ss = og.SimpleSetup(space)
    # set function to check if a state is valid
    ss.setStateValidityChecker(ob.StateValidityCheckerFn(partial(isStateValid, ss.getSpaceInformation())))

    # set start state
    start = ob.State(space)
    start().setX(START_LOC[0])
    start().setY(START_LOC[1])
    # set goal state
    goal = ob.State(space)
    goal().setX(GOAL_LOC[0])
    goal().setY(GOAL_LOC[1])
    # apply these start and goal states to the problem
    ss.setStartAndGoalStates(start, goal, TOLERANCE)

    si = ss.getSpaceInformation()
    # use RRT as the planning
    planner = og.RRT(si)
    ss.setPlanner(planner)

    # solve problem (limit of 600 seconds)
    solved = ss.solve(600.0)

    # retrieve info about the solution path if it exists
    path_x = []
    path_y = []
    path_xy = []
    if solved:
        print('Path found')
        # get solution path
        solPath = ss.getSolutionPath()
        path = solPath.getStates()
        # get states in path to visualize
        for p in path:
            path_x.append(p.getX())
            path_y.append(p.getY())
            path_xy.append([p.getX(), p.getY()])
    else:
        print('No path found')
        
    return path_x, path_y, np.array(path_xy)

In [26]:
# run planning algorithm
path_x, path_y, path_xy = plan()

Path found
Debug:   RRT: Planner range detected to be 0.879845
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 36 states
Info:    Solution found in 0.001959 seconds


In [ ]:
# visualize the solution path
_, ax = plt.subplots()

# Start and end goal
start_point = plt.Circle((START_LOC[0], START_LOC[1]), TOLERANCE, color='green')
goal_point = plt.Circle((GOAL_LOC[0], GOAL_LOC[1]), TOLERANCE, color='red')
ax.add_patch(start_point)
ax.add_patch(goal_point)

# Path
ax.plot(path_x, path_y, color='blue')
plt.show()

As you can see, we are quite easily able to construct a planning problem and find a path from start to finish.  The last code block visualizes the path using matplotlib.  The start location is shown in green and the goal is in red.  The size of the circle is based on how close the robot has to be to count as reaching it.  This value can be changed using the TOLERANCE variable.

# Adding obstacles
Now we will add collision detection and obstcles.  These will be invalid positions that the robot must avoid.  We will edit the state validity function to detect for collision with obstacles.
![title](images/obstacles.png)

In [ ]:
# all the imports we'll need for this OMPL tutorial
from math import sin, cos
from functools import partial
import sys
from os.path import abspath, dirname, join
import sys
sys.path.insert(0, join(abspath(''), 'py-bindings'))
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og
import data
import math
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [51]:
# obstacles to avoid
# each row is a single obstacle
# the first element is the bottom left coordinate and the second is the top right coordinate
OBSTACLES = [
    [[-1, -0.3], [0, -0.2]],
    [[0, 0.2], [1, 0.3]]
]

In [52]:
# function to check distance between a line segment and a point
# helper function used in collision detection
def distPointToLine(pt, lp1, lp2):
    return dist(lp1[0], lp1[1], lp2[0], lp2[1], pt[0], pt[1])

def dist(x1, y1, x2, y2, x3, y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1
    norm = px*px + py*py
    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(norm)
    if u > 1:
        u = 1
    elif u < 0:
        u = 0
    x = x1 + u * px
    y = y1 + u * py
    dx = x - x3
    dy = y - y3
    dist = (dx*dx + dy*dy)**.5
    return dist

In [53]:
# define the function to check if a state is valid
MARGIN = 0.05
def isStateValid(spaceInformation, state):
    sx = state.getX()
    sy = state.getY()
    
    for obs in OBSTACLES:
        if sx >= obs[0][0] and sx <= obs[1][0] and sy >= obs[0][1] and sy <= obs[1][1]:
            return False
    for obs in OBSTACLES:
        if ( distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[1][0], obs[0][1]] # bottom line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[0][0], obs[1][1]] # left line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[1][0], obs[0][1]], [obs[1][0], obs[1][1]] # right line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[1][1]], [obs[1][0], obs[1][1]] # top line
            ) < MARGIN
           ):
            return False
    return spaceInformation.satisfiesBounds(state)

In [54]:
TOLERANCE = 0.1
START_LOC = (-0.8, -0.8)
GOAL_LOC = (0.8, 0.8)
BOUNDS = (-1, 1)

# main function to plan path
def plan():
    # construct the SE2 state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    # set highest and lowest values for each vector component
    bounds.setLow(BOUNDS[0])
    bounds.setHigh(BOUNDS[1])
    space.setBounds(bounds)

    # handle setup for planning
    ss = og.SimpleSetup(space)
    # set function to check if a state is valid
    ss.setStateValidityChecker(ob.StateValidityCheckerFn(partial(isStateValid, ss.getSpaceInformation())))

    # set start state
    start = ob.State(space)
    start().setX(START_LOC[0])
    start().setY(START_LOC[1])
    # set goal state
    goal = ob.State(space)
    goal().setX(GOAL_LOC[0])
    goal().setY(GOAL_LOC[1])
    # apply these start and goal states to the problem
    ss.setStartAndGoalStates(start, goal, TOLERANCE)

    si = ss.getSpaceInformation()
    # use RRT as the planning
    planner = og.RRT(si)
    ss.setPlanner(planner)

    # solve problem (limit of 600 seconds)
    solved = ss.solve(600.0)

    # retrieve info about the solution path if it exists
    path_x = []
    path_y = []
    path_xy = []
    if solved:
        print('Path found')
        # get solution path
        solPath = ss.getSolutionPath()
        path = solPath.getStates()
        # get states in path to visualize
        for p in path:
            path_x.append(p.getX())
            path_y.append(p.getY())
            path_xy.append([p.getX(), p.getY()])
    else:
        print('No path found')
        
    return path_x, path_y, np.array(path_xy)

In [55]:
# run planning algorithm
path_x, path_y, path_xy = plan()

Debug:   RRT: Planner range detected to be 0.879845
Info:    RRT: Starting planning with 1 states already in datastructure
Path found
Info:    RRT: Created 86 states
Info:    Solution found in 0.023202 seconds


In [ ]:
# visualize the solution path
_, ax = plt.subplots()
plt.xlim(BOUNDS[0], BOUNDS[1])
plt.ylim(BOUNDS[0], BOUNDS[1])

# Obstacles
for d in OBSTACLES:
    x = [ d[0][0] - MARGIN, d[0][0] - MARGIN, d[1][0] + MARGIN, d[1][0] + MARGIN, d[0][0] - MARGIN ]
    y = [ d[0][1] - MARGIN, d[1][1] + MARGIN, d[1][1] + MARGIN, d[0][1] - MARGIN, d[0][1] - MARGIN ]
    ax.fill(x, y, color='orange')
for d in OBSTACLES:
    x = [ d[0][0], d[0][0], d[1][0], d[1][0], d[0][0] ]
    y = [ d[0][1], d[1][1], d[1][1], d[0][1], d[0][1] ]
    ax.fill(x, y, color='black')

# Start and end goal
start_point = plt.Circle((START_LOC[0], START_LOC[1]), TOLERANCE, color='green')
goal_point = plt.Circle((GOAL_LOC[0], GOAL_LOC[1]), TOLERANCE, color='red')
ax.add_patch(start_point)
ax.add_patch(goal_point)

# Path
ax.plot(path_x, path_y, color='blue')
plt.show()

You see that we can now force the robot to avoid certain areas.  This will be useful when we add in the maze.  Again, the start is green, the goal is red, and the path is blue.  The black are the obstacles.  The orange is a margin value that states how far away from the obstacles the robot should stay.  This value can be changed with the MARGIN variable.

# Adding motion constraints
Now we will add motion constraints to the agent so that it doesn't have a finite turning radius.  This will be accomplished by creating a control space and dealing with yaw components of the robot's state.  We will also add a state propagator function which will allow us to calculate what the next state will be.
![title](images/constraints.png)

In [ ]:
# all the imports we'll need for this OMPL tutorial
from math import sin, cos
from functools import partial
import sys
from os.path import abspath, dirname, join
import sys
sys.path.insert(0, join(abspath(''), 'py-bindings'))
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og
import data
import math
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [67]:
# obstacles to avoid
# each row is a single obstacle
# the first element is the bottom left coordinate and the second is the top right coordinate
OBSTACLES = [
    [[-1, -0.3], [0, -0.2]],
    [[0, 0.2], [1, 0.3]]
]

In [68]:
# function to check distance between a line segment and a point
# helper function used in collision detection
def distPointToLine(pt, lp1, lp2):
    return dist(lp1[0], lp1[1], lp2[0], lp2[1], pt[0], pt[1])

def dist(x1, y1, x2, y2, x3, y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1
    norm = px*px + py*py
    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(norm)
    if u > 1:
        u = 1
    elif u < 0:
        u = 0
    x = x1 + u * px
    y = y1 + u * py
    dx = x - x3
    dy = y - y3
    dist = (dx*dx + dy*dy)**.5
    return dist

In [69]:
# state propagater to transform to the next state
# takes in the state and a control signal on how to move
def propagate(start, control, duration, state):
    state.setX(start.getX() + control[0] * duration * cos(start.getYaw()))
    state.setY(start.getY() + control[0] * duration * sin(start.getYaw()))
    state.setYaw(start.getYaw() + control[1] * duration)

In [70]:
# define the function to check if a state is valid
MARGIN = 0.05
def isStateValid(spaceInformation, state):
    sx = state.getX()
    sy = state.getY()
    
    for obs in OBSTACLES:
        if sx >= obs[0][0] and sx <= obs[1][0] and sy >= obs[0][1] and sy <= obs[1][1]:
            return False
    for obs in OBSTACLES:
        if ( distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[1][0], obs[0][1]] # bottom line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[0][0], obs[1][1]] # left line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[1][0], obs[0][1]], [obs[1][0], obs[1][1]] # right line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[1][1]], [obs[1][0], obs[1][1]] # top line
            ) < MARGIN
           ):
            return False
    return spaceInformation.satisfiesBounds(state)

In [71]:
TOLERANCE = 0.1
# deine max turning angle
MAX_YAW = 0.2
# add yaw component to start and goal locations
START_LOC = (-0.8, -0.8, 0)
GOAL_LOC = (0.8, 0.8, 0)
BOUNDS = (-1, 1)

# main function to plan path
def plan():
    # construct the SE2 state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    # set highest and lowest values for each vector component
    bounds.setLow(BOUNDS[0])
    bounds.setHigh(BOUNDS[1])
    space.setBounds(bounds)
    
    # create a control space that defines how the robot can move
    cspace = oc.RealVectorControlSpace(space, 2)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(2)
    cbounds.setLow(-MAX_YAW)
    cbounds.setHigh(MAX_YAW)
    cspace.setBounds(cbounds)

    # handle setup for planning
    # note the change from og to oc to denote the use of the control module
    ss = oc.SimpleSetup(cspace)
    # set function to check if a state is valid
    ss.setStateValidityChecker(ob.StateValidityCheckerFn(partial(isStateValid, ss.getSpaceInformation())))
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))

    # set start state
    start = ob.State(space)
    start().setX(START_LOC[0])
    start().setY(START_LOC[1])
    # set goal state
    goal = ob.State(space)
    goal().setX(GOAL_LOC[0])
    goal().setY(GOAL_LOC[1])
    # apply these start and goal states to the problem
    ss.setStartAndGoalStates(start, goal, TOLERANCE)

    si = ss.getSpaceInformation()
    # use RRT as the planning
    # switch to using oc.RRT so that it will obey the constraints
    planner = oc.RRT(si)
    ss.setPlanner(planner)
    # set how much the propagater can change
    si.setPropagationStepSize(.1)

    # solve problem (limit of 600 seconds)
    solved = ss.solve(600.0)

    # retrieve info about the solution path if it exists
    path_x = []
    path_y = []
    path_xy = []
    if solved:
        print('Path found')
        # get solution path
        solPath = ss.getSolutionPath()
        path = solPath.getStates()
        # get states in path to visualize
        for p in path:
            path_x.append(p.getX())
            path_y.append(p.getY())
            path_xy.append([p.getX(), p.getY()])
    else:
        print('No path found')
        
    return path_x, path_y, np.array(path_xy)

In [72]:
# run planning algorithm
path_x, path_y, path_xy = plan()

Info:    RRT: Starting planning with 1 states already in datastructure


         at line 77 in /home/cforsyth/Downloads/ompl-1.5.2/src/ompl/control/src/SpaceInformation.cpp


Path found
Info:    RRT: Created 13240 states
Info:    Solution found in 2.081780 seconds


In [ ]:
# visualize the solution path
_, ax = plt.subplots()
plt.xlim(BOUNDS[0], BOUNDS[1])
plt.ylim(BOUNDS[0], BOUNDS[1])

# Obstacles
for d in OBSTACLES:
    x = [ d[0][0] - MARGIN, d[0][0] - MARGIN, d[1][0] + MARGIN, d[1][0] + MARGIN, d[0][0] - MARGIN ]
    y = [ d[0][1] - MARGIN, d[1][1] + MARGIN, d[1][1] + MARGIN, d[0][1] - MARGIN, d[0][1] - MARGIN ]
    ax.fill(x, y, color='orange')
for d in OBSTACLES:
    x = [ d[0][0], d[0][0], d[1][0], d[1][0], d[0][0] ]
    y = [ d[0][1], d[1][1], d[1][1], d[0][1], d[0][1] ]
    ax.fill(x, y, color='black')

# Start and end goal
start_point = plt.Circle((START_LOC[0], START_LOC[1]), TOLERANCE, color='green')
goal_point = plt.Circle((GOAL_LOC[0], GOAL_LOC[1]), TOLERANCE, color='red')
ax.add_patch(start_point)
ax.add_patch(goal_point)

# Path
ax.plot(path_x, path_y, color='blue')
plt.show()

You can see that now the robot smoothely turns to change direction and navigate to the goal.  The degree to which the robot can turn can be modified with the MAX_YAW variable.  One thing you will note here is that for the middle segment of the trip, the robot is driving backwards.  For our purposes here, we will ignore this, but it could be extra work if desired.  A potential solution to getting the robot to not drive backwards for a very long time would be track how much the robot is moving backwards each time it does and then once it passes a certain threshhodl, all backward driving states are counted as invalid until it drives forward again.

# PyBullet Simulation
Now that most of the OMPL problem has been set up, we will switch to working with PyBullet to visualize and simulate the planning for a snake robot.

# Rendering
The first part will be to render a maze like area that will eventually be used to plan in.  This is to make sure that we are able to correctly visualize environments.  PyBullet uses a server-client to do computations and so that is why there is code to connect to something.
![title](images/rendering.png)

In [1]:
# all the needed imports for the PyBullet code
import time
import math
import pybullet as p
import pybullet_data
from data import WORLD_OFFSET_X, WORLD_OFFSET_Y, CELL_SIZE, SCALE, OBS_HEIGHT, OBSTACLES, WAYPOINTS
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [2]:
# takes in two points and will create an obstacle at the given location
# the first point values are the lower left corner and the second are the upper right
# OBS_HEIGHT, WORLD_OFFSET_X, WORLD_OFFSET_Y, and SCALE are defined in the data.py file
# these are used to set hte obstacle height, offset of the displayed environment and its scale
def createObstacle(x1, y1, x2, y2):
    z1, z2 = 0, OBS_HEIGHT
    x1, y1, x2, y2 = x1 + WORLD_OFFSET_X, y1 + WORLD_OFFSET_Y, x2 + WORLD_OFFSET_X, y2 + WORLD_OFFSET_Y
    x1, y1, z1, x2, y2, z2 = x1 * SCALE, y1 * SCALE, z1 * SCALE, x2 * SCALE, y2 * SCALE, z2 * SCALE
    we, le, he = (x2 - x1) / 2, (y2 - y1) / 2, (z2 - z1) / 2
    x, y, z = x1 + we, y1 + le, z1 + he
    boxId = p.createCollisionShape(p.GEOM_BOX, halfExtents=[we, le, he])
    boxMB = p.createMultiBody(
        0,
        boxId,
        -1,
        [x, y, z],
        [0, 0, 0, 1]
    )

In [ ]:
# connect and visually display environment
p.connect(p.GUI)
# be able to access the pybullet_data module
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# set rotation and position of the camera
p.resetDebugVisualizerCamera(15, 0, -89, [0, 0, 1])

# ground plane
plane = p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, plane)

# do not run simulation in real time, wait for stepSimulation to be called
p.setRealTimeSimulation(0)

# create obstacles for all the ones in the list
for obs in OBSTACLES:
    createObstacle(obs[0][0], obs[0][1], obs[1][0], obs[1][1])
    
dt = 1. / 240.
while True:
    p.stepSimulation()
    time.sleep(dt)

# Snake Modeling
Another main part of the simulation is getting the snake robot to actually render and behavior as it should.  Here we use a modified version of the snake demo from PyBullet and are able to control it using WASD.  We won't add the obstacles in for clarity.  The snake will be a set of connected boxes.
![title](images/snake.png)

In [1]:
# all the needed imports for the PyBullet code
import time
import math
import pybullet as p
import pybullet_data
from data import WORLD_OFFSET_X, WORLD_OFFSET_Y, CELL_SIZE, SCALE, OBS_HEIGHT, OBSTACLES, WAYPOINTS
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [ ]:
# connect to GUI
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# create ground
plane = p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, plane)

useMaximalCoordinates = True
# half side length of each box segment of the snake
halfBoxLen = 0.25
# create a box
# when you create something, an index is returned that can be used to refer to the object
colBoxId = p.createCollisionShape(p.GEOM_BOX, halfExtents=[halfBoxLen, halfBoxLen, halfBoxLen])

link_Masses = [] # mass of each segment
linkCollisionShapeIndices = [] # id of collision shape for each segment
linkVisualShapeIndices = [] # id of visual object for each segment
linkPositions = [] # position of links (joint) to connect segments
linkOrientations = [] # orientation of links (joint) to connect segments
linkInertialFramePositions = [] # link inertial frame positions for each join
linkInertialFrameOrientations = [] # link inertial frame orientation for each joint
indices = [] # indices of each component of the snake
jointTypes = [] # types of each link
axis = [] # axis of each link

# create 6 more segments
# in the next part, the main segment is created so this makes a total of 7 segments
for i in range(6):
    link_Masses.append(1) # mass = 1
    linkCollisionShapeIndices.append(colBoxId) # each segment uses same collision template
    linkVisualShapeIndices.append(-1) # no visual object added (use default coloring)
    linkPositions.append([0, halfBoxLen * 2.0 + 0.01, 0]) # offset each next link relative to the previous
    linkOrientations.append([0, 0, 0, 1]) # set link orientations
    linkInertialFramePositions.append([0, 0, 0]) # set link inertial frame positions
    linkInertialFrameOrientations.append([0, 0, 0, 1]) # set link inertial frame orientations
    indices.append(i) # add indices of components
    jointTypes.append(p.JOINT_REVOLUTE) # all joints are of type JOINT_REVOLUTE
    axis.append([0, 0, 1]) # each joint revolves around z axis

# head initial position and orientation
basePosition = [0, 0, 1]
baseOrientation = [0, 0, 0, 1]
snakeId = p.createMultiBody(1,
                              colBoxId,
                              -1,
                              basePosition,
                              baseOrientation,
                              linkMasses=link_Masses,
                              linkCollisionShapeIndices=linkCollisionShapeIndices,
                              linkVisualShapeIndices=linkVisualShapeIndices,
                              linkPositions=linkPositions,
                              linkOrientations=linkOrientations,
                              linkInertialFramePositions=linkInertialFramePositions,
                              linkInertialFrameOrientations=linkInertialFrameOrientations,
                              linkParentIndices=indices,
                              linkJointTypes=jointTypes,
                              linkJointAxis=axis,
                              useMaximalCoordinates=useMaximalCoordinates)

# set gravity to pull down on z axis
p.setGravity(0, 0, -10)
# don't simulate without stepSimulation being called
p.setRealTimeSimulation(0)

anistropicFriction = [1, 0.01, 0.01]
p.changeDynamics(colBoxId, -1, lateralFriction=2, anisotropicFriction=anistropicFriction)
p.getNumJoints(colBoxId)
for i in range(p.getNumJoints(snakeId)):
    p.getJointInfo(snakeId, i)
    p.changeDynamics(snakeId, i, lateralFriction=2, anisotropicFriction=anistropicFriction)

# time between each simulation step
dt = 1. / 240.
SNAKE_NORMAL_PERIOD = 0.1  #1.5
m_wavePeriod = SNAKE_NORMAL_PERIOD

# value to change how snake acts
m_waveLength = 4
m_wavePeriod = 1.5
m_waveAmplitude = 0.4
m_waveFront = 0.0
#our steering value
m_steering = 0.0
m_segmentLength = halfBoxLen * 2.0
forward = 0

while (1):
    # keyboard UP, DOWN, LEFT, and RIGHT arrows can be used to control the snake
    keys = p.getKeyboardEvents()
    for k, v in keys.items():
        if (k == p.B3G_RIGHT_ARROW and (v & p.KEY_WAS_TRIGGERED)):
            m_steering = .2
        if (k == p.B3G_RIGHT_ARROW and (v & p.KEY_WAS_RELEASED)):
            m_steering = 0
        if (k == p.B3G_LEFT_ARROW and (v & p.KEY_WAS_TRIGGERED)):
            m_steering = -.2
        if (k == p.B3G_LEFT_ARROW and (v & p.KEY_WAS_RELEASED)):
            m_steering = 0

    amp = 0.2
    offset = 0.6
    numMuscles = p.getNumJoints(snakeId)
    scaleStart = 1.0

    #start of the snake with smaller waves.
    if (m_waveFront < m_segmentLength * 4.0):
        scaleStart = m_waveFront / (m_segmentLength * 4.0)

    segment = numMuscles - 1

    #we simply move a sin wave down the body of the snake.
    # move each joint using steering signal
    for joint in range(p.getNumJoints(snakeId)):
        segment = joint  #numMuscles-1-joint
        #map segment to phase
        phase = (m_waveFront - (segment + 1) * m_segmentLength) / m_waveLength
        phase -= math.floor(phase)
        phase *= math.pi * 2.0

        #map phase to curvature
        targetPos = math.sin(phase) * scaleStart * m_waveAmplitude

        # steer snake by squashing +ve or -ve side of sin curve
        if (m_steering > 0 and targetPos < 0):
            targetPos *= 1.0 / (1.0 + m_steering)

        if (m_steering < 0 and targetPos > 0):
            targetPos *= 1.0 / (1.0 - m_steering)

        #set our motor
        p.setJointMotorControl2(snakeId,
                                joint,
                                p.POSITION_CONTROL,
                                targetPosition=targetPos + m_steering,
                                force=30)

    #wave keeps track of where the wave is in time
    m_waveFront += dt / m_wavePeriod * m_waveLength
    # step simulation forward
    p.stepSimulation()

    # wait until next step should be taken
    time.sleep(dt)

So now the snake should work properly.  You can zoom in and out with the scroll wheel on the mouse and can look around by holding down ALT and left click and drag with the mouse.  One of the difficulties with controlling the snake model is that you cannot always immediately turn in whichever direction you want to.  Because the snake has a periodic movement, you can only quickly and easily turn left or right at certain times of the movement.

# Waypoint following
The OMPL aspect of the program will be the one to plan and generate waypoints using RRT.  These waypoints will denote major points in the path from the start to the goal.  These waypoints will be passed to the PyBullet aspect which will use a PD controller to guide the snake to follow these points to the goal.
In this seciton, we will implement a waypoint following controller for the snake from the previous section.  Again we will have an empty environment aside from the ground plane.  For now we will manually create waypoints.
![title](images/following.png)

In [1]:
# all the needed imports for the PyBullet code
import time
import math
import pybullet as p
import pybullet_data
from data import WORLD_OFFSET_X, WORLD_OFFSET_Y, CELL_SIZE, SCALE, OBS_HEIGHT, OBSTACLES, WAYPOINTS
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [2]:
# calculate distance between two points
def distance(pt1, pt2):
    return math.sqrt(
        pow(pt1[0] - pt2[0], 2) +
        pow(pt1[1] - pt2[1], 2)
    )

In [3]:
# adjust angle so 0 is directly to the right
def fix_angle(angle):
    angle += 10 * math.pi
    while math.fabs(angle) > math.pi:
        angle -= (2 * math.pi)
    return angle

In [4]:
# list of waypoints to follow
val = 10
waypoints = [
    (-val, -val),
    (-val, val),
    (val, val),
    (val, -val),
    (0, 0)
]
# track which waypoint currently heading towards
waypoint_index = 0

In [ ]:
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# create ground
plane = p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, plane)

useMaximalCoordinates = True
# half side length of each box segment of the snake
halfBoxLen = 0.25
# create a box
# when you create something, an index is returned that can be used to refer to the object
colBoxId = p.createCollisionShape(p.GEOM_BOX, halfExtents=[halfBoxLen, halfBoxLen, halfBoxLen])

link_Masses = [] # mass of each segment
linkCollisionShapeIndices = [] # id of collision shape for each segment
linkVisualShapeIndices = [] # id of visual object for each segment
linkPositions = [] # position of links (joint) to connect segments
linkOrientations = [] # orientation of links (joint) to connect segments
linkInertialFramePositions = [] # link inertial frame positions for each join
linkInertialFrameOrientations = [] # link inertial frame orientation for each joint
indices = [] # indices of each component of the snake
jointTypes = [] # types of each link
axis = [] # axis of each link

# create 6 more segments
# in the next part, the main segment is created so this makes a total of 7 segments
for i in range(6):
    link_Masses.append(1) # mass = 1
    linkCollisionShapeIndices.append(colBoxId) # each segment uses same collision template
    linkVisualShapeIndices.append(-1) # no visual object added (use default coloring)
    linkPositions.append([0, halfBoxLen * 2.0 + 0.01, 0]) # offset each next link relative to the previous
    linkOrientations.append([0, 0, 0, 1]) # set link orientations
    linkInertialFramePositions.append([0, 0, 0]) # set link inertial frame positions
    linkInertialFrameOrientations.append([0, 0, 0, 1]) # set link inertial frame orientations
    indices.append(i) # add indices of components
    jointTypes.append(p.JOINT_REVOLUTE) # all joints are of type JOINT_REVOLUTE
    axis.append([0, 0, 1]) # each joint revolves around z axis

# head initial position and orientation
basePosition = [0, 0, 1]
baseOrientation = [0, 0, 0, 1]
snakeId = p.createMultiBody(1,
                              colBoxId,
                              -1,
                              basePosition,
                              baseOrientation,
                              linkMasses=link_Masses,
                              linkCollisionShapeIndices=linkCollisionShapeIndices,
                              linkVisualShapeIndices=linkVisualShapeIndices,
                              linkPositions=linkPositions,
                              linkOrientations=linkOrientations,
                              linkInertialFramePositions=linkInertialFramePositions,
                              linkInertialFrameOrientations=linkInertialFrameOrientations,
                              linkParentIndices=indices,
                              linkJointTypes=jointTypes,
                              linkJointAxis=axis,
                              useMaximalCoordinates=useMaximalCoordinates)

p.setGravity(0, 0, -10)
p.setRealTimeSimulation(0)

anistropicFriction = [1, 0.01, 0.01]
p.changeDynamics(snakeId, -1, lateralFriction=2, anisotropicFriction=anistropicFriction)
p.getNumJoints(snakeId)
for i in range(p.getNumJoints(snakeId)):
    p.getJointInfo(snakeId, i)
    p.changeDynamics(snakeId, i, lateralFriction=2, anisotropicFriction=anistropicFriction)

dt = 1. / 240.
SNAKE_NORMAL_PERIOD = 0.1
m_wavePeriod = SNAKE_NORMAL_PERIOD

STEERING_CONST = 0.2

m_waveLength = 4
m_wavePeriod = 1.5
m_waveAmplitude = 0.4
m_waveFront = 0.0
#our steering value
m_steering = 0.0
m_segmentLength = halfBoxLen * 2.0

# visualize points where each waypoint is
for w in waypoints:
    wid = p.createVisualShape(
        p.GEOM_SPHERE,
        radius=0.2,
        rgbaColor=[0, 0, 0, 1],
    )
    wmb = p.createMultiBody(
        0,
        -1,
        wid,
        [w[0], w[1], 0.25]
    )

TOLERANCE = 0.1
# run until the last waypoint has been reached
while (waypoint_index < len(waypoints)):
    base_state = p.getBasePositionAndOrientation(snakeId)
    # location of robot
    robot_pos = base_state[0]
    robot_orien = p.getEulerFromQuaternion(base_state[1])
    # angle robot is facing
    robot_angle = robot_orien[2]
    robot_angle -= math.pi / 2
    robot_angle = fix_angle(robot_angle)

    # location of next waypoint
    goal_pos = waypoints[waypoint_index]

    # if at the waypoint, head to the next one
    if distance(robot_pos, goal_pos) < TOLERANCE:
        waypoint_index += 1
    
    ray = (goal_pos[0] - robot_pos[0], goal_pos[1] - robot_pos[1])
    goal_angle = math.atan2(ray[1], ray[0])
    goal_angle = fix_angle(goal_angle)

    # relative angle to next waypoint from the robot
    rel_angle = goal_angle - robot_angle
    rel_angle = fix_angle(rel_angle)

    # controller to turn towards the waypoint
    if rel_angle > 0:
        m_steering = -STEERING_CONST
    else:
        m_steering = STEERING_CONST

    numMuscles = p.getNumJoints(snakeId)
    scaleStart = 1.0

    #start of the snake with smaller waves.
    if (m_waveFront < m_segmentLength * 4.0):
        scaleStart = m_waveFront / (m_segmentLength * 4.0)

    segment = numMuscles - 1

    #we simply move a sin wave down the body of the snake.c
    for joint in range(p.getNumJoints(snakeId)):
        segment = joint  #numMuscles-1-joint
        #map segment to phase
        phase = (m_waveFront - (segment + 1) * m_segmentLength) / m_waveLength
        phase -= math.floor(phase)
        phase *= math.pi * 2.0

        #map phase to curvature
        targetPos = math.sin(phase) * scaleStart * m_waveAmplitude

        # steer snake by squashing +ve or -ve side of sin curve
        if (m_steering > 0 and targetPos < 0):
            targetPos *= 1.0 / (1.0 + m_steering)

        if (m_steering < 0 and targetPos > 0):
            targetPos *= 1.0 / (1.0 - m_steering)

        #set our motor
        p.setJointMotorControl2(
            snakeId,
            joint,
            p.POSITION_CONTROL,
            targetPosition=targetPos + m_steering,
            force=30
        )

    #wave keeps track of where the wave is in time
    m_waveFront += dt / m_wavePeriod * m_waveLength
    p.stepSimulation()

    time.sleep(dt)

Running this program you can see the black spheres represent the waypoints in the list.  The snake follows each of them and once it reaches one, it moves on to the next one.  We have all the pieces now that we need to create an integrated OMPL and PyBullet program.

# Putting it all together
Now we will combine what we did in the OMPL section with what we did in the PyBullet section to create a final program that will read in the obstacles data from data.py.  It will create the constrained space of the snake robot.  It will use RRT to plan and find a path from the start location to the goal location while respecting the boundaries of the obstacles and the constraints on motion from the dynamics of the snake robot.  Once it has run and found a path, it will extract out the waypoints and deposit them to data files like waypoints.data, waypoints_10.data, and waypoints_50.data.  The difference between these files is the first has all waypoints (approx 1400), the second has every tenth point (approx 140), and the final has every 50th point (approx 28).  The full path will visualized int he 2D chart created using matplotlib.
Then PyBullet will read in these waypoints.  We are using waypoints_50.data for performance reasons.  PyBullet will construct the walls, the snake, and the waypoints as static collision objects, dynamic collision objects, and collisionless visual objects respectively.  Then it will run the simulation and show that the generated path works in the simulated environment.

### Planning
![title](images/ompl.png)

In [1]:
from math import sin, cos
from functools import partial
import sys
from os.path import abspath, dirname, join
import sys
sys.path.insert(0, join(abspath(''), 'py-bindings'))
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og
import data
import math
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [2]:
MAX_YAW = 0.2
MARGIN = 0.5
TOLERANCE = 0.05
start_loc = (11.5, 9, -math.pi / 2)
goal_loc = (1.5, 18.5, math.pi / 2)

In [3]:
# find the distance between a point and a line
def distPointToLine(pt, lp1, lp2):
    return dist(lp1[0], lp1[1], lp2[0], lp2[1], pt[0], pt[1])
def dist(x1, y1, x2, y2, x3, y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1
    norm = px*px + py*py
    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(norm)
    if u > 1:
        u = 1
    elif u < 0:
        u = 0
    x = x1 + u * px
    y = y1 + u * py
    dx = x - x3
    dy = y - y3
    dist = (dx*dx + dy*dy)**.5
    return dist

In [4]:
# checks if a state is valid
# checks against bounds and obstacles
def isStateValid(spaceInformation, state):
    sx = state.getX()
    sy = state.getY()
    # check if snake is inside an obstacle
    for obs in data.OBSTACLES:
        if sx >= obs[0][0] and sx <= obs[1][0] and sy >= obs[0][1] and sy <= obs[1][1]:
            return False
    # check if snake is too close to edge of an obstacle
    for obs in data.OBSTACLES:
        if ( distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[1][0], obs[0][1]] # bottom line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[0][1]], [obs[0][0], obs[1][1]] # left line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[1][0], obs[0][1]], [obs[1][0], obs[1][1]] # right line
            ) < MARGIN or
            distPointToLine(
                [sx, sy], [obs[0][0], obs[1][1]], [obs[1][0], obs[1][1]] # top line
            ) < MARGIN
           ):
            return False
    # check if bounds are satisfied
    return spaceInformation.satisfiesBounds(state)


In [5]:
# state propagater, calculates a new state from the current state
def propagate(start, control, duration, state):
    state.setX(start.getX() + (control[0] + 0.1) * duration * cos(start.getYaw()))
    state.setY(start.getY() + (control[0] + 0.1) * duration * sin(start.getYaw()))
    state.setYaw(start.getYaw() + control[1] * duration)

In [6]:
# function to plan
def plan():
    # construct the state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(-20)
    bounds.setHigh(20)
    space.setBounds(bounds)
    # create a control space
    cspace = oc.RealVectorControlSpace(space, 2)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(2)
    cbounds.setLow(-MAX_YAW)
    cbounds.setHigh(MAX_YAW)
    cspace.setBounds(cbounds)

    # set up problem, state validity checker, and state propagater
    ss = oc.SimpleSetup(cspace)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))

    # define start state
    start = ob.State(space)
    start().setX(start_loc[0])
    start().setY(start_loc[1])
    start().setYaw(start_loc[2])

    # define goal state
    goal = ob.State(space)
    goal().setX(goal_loc[0])
    goal().setY(goal_loc[1])
    goal().setYaw(goal_loc[2])
    
    # set the start and goal states
    print('start')
    print(start)
    print('goal')
    print(goal)
    print()
    ss.setStartAndGoalStates(start, goal, TOLERANCE)

    # set up RRT to use the control space
    si = ss.getSpaceInformation()
    planner = oc.RRT(si)
    ss.setPlanner(planner)
    si.setPropagationStepSize(.1)

    # solve problem
    solved = ss.solve(600.0)

    # export path
    path_x = []
    path_y = []
    path_xy = []
    if solved:
        solPath = ss.getSolutionPath()
        path = solPath.getStates()
        for p in path:
            path_x.append(p.getX())
            path_y.append(p.getY())
            path_xy.append([p.getX(), p.getY()])
        
    return path_x, path_y, np.array(path_xy)

In [7]:
path_x, path_y, path_xy = plan()

start
Compound state [
RealVectorState [11.5 9]
SO2State [-1.5708]
]

goal
Compound state [
RealVectorState [1.5 18.5]
SO2State [1.5708]
]


Info:    RRT: Starting planning with 1 states already in datastructure


         at line 77 in /home/cforsyth/Downloads/ompl-1.5.2/src/ompl/control/src/SpaceInformation.cpp


Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 77040 states
Info:    Solution found in 600.020016 seconds


In [ ]:
_, ax = plt.subplots()

# draw obstacles
for d in data.OBSTACLES:
    x = [ d[0][0] - MARGIN, d[0][0] - MARGIN, d[1][0] + MARGIN, d[1][0] + MARGIN, d[0][0] - MARGIN ]
    y = [ d[0][1] - MARGIN, d[1][1] + MARGIN, d[1][1] + MARGIN, d[0][1] - MARGIN, d[0][1] - MARGIN ]
    ax.fill(x, y, color='orange')
for d in data.OBSTACLES:
    x = [ d[0][0], d[0][0], d[1][0], d[1][0], d[0][0] ]
    y = [ d[0][1], d[1][1], d[1][1], d[0][1], d[0][1] ]
    ax.fill(x, y, color='black')

# draw start and end goal
start_point = plt.Circle((start_loc[0], start_loc[1]), TOLERANCE, color='green')
goal_point = plt.Circle((goal_loc[0], goal_loc[1]), TOLERANCE, color='red')
ax.add_patch(start_point)
ax.add_patch(goal_point)

# draw path
ax.plot(path_x, path_y, color='blue')
    
plt.show()

In [9]:
# break down path into subsections
path_xy_10 = []
path_xy_50 = []
for i, val in enumerate(path_xy):
    if i % 10 == 0:
        path_xy_10.append(val)
    if i % 50 == 0:
        path_xy_50.append(val)
        
# save waypoints to file
file = open('waypoints.data', "wb")
np.save(file, path_xy)
file.close()

file_10 = open('waypoints_10.data', 'wb')
np.save(file_10, path_xy_10)
file_10.close()

file_50 = open('waypoints_50.data', 'wb')
np.save(file_50, path_xy_50)
file_50.close()

### Simulation
![title](images/pybullet.png)

In [10]:
import time
import math
import pybullet as p
import pybullet_data
from data import WORLD_OFFSET_X, WORLD_OFFSET_Y, CELL_SIZE, SCALE, OBS_HEIGHT, OBSTACLES, WAYPOINTS
import numpy as np

pybullet build time: May 20 2022 19:44:17


In [11]:
# distance between two points used to find distance between snake and waypoints
def distance(pt1, pt2):
    return math.sqrt(
        pow(pt1[0] - pt2[0], 2) +
        pow(pt1[1] - pt2[1], 2)
    )

In [12]:
# readjusts angles so that 0 is directly to the right
def fix_angle(angle):
    angle += 10 * math.pi
    while math.fabs(angle) > math.pi:
        angle -= (2 * math.pi)
    return angle

In [13]:
# creates a collision object 
def createObstacle(x1, y1, x2, y2):
    z1, z2 = 0, OBS_HEIGHT
    x1, y1, x2, y2 = x1 + WORLD_OFFSET_X, y1 + WORLD_OFFSET_Y, x2 + WORLD_OFFSET_X, y2 + WORLD_OFFSET_Y
    x1, y1, z1, x2, y2, z2 = x1 * SCALE, y1 * SCALE, z1 * SCALE, x2 * SCALE, y2 * SCALE, z2 * SCALE
    we, le, he = (x2 - x1) / 2, (y2 - y1) / 2, (z2 - z1) / 2
    x, y, z = x1 + we, y1 + le, z1 + he
    boxId = p.createCollisionShape(p.GEOM_BOX, halfExtents=[we, le, he])
    boxMB = p.createMultiBody(
        0,
        boxId,
        -1,
        [x, y, z],
        [0, 0, 0, 1]
    )

In [14]:
# distance from waypoint to count as reaching it.
TOLERANCE = 0.5

In [15]:
# connect to GUI
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# setup camera
p.resetDebugVisualizerCamera(15, 0, -89, [0, 0, 1])

# create ground
plane = p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, plane)

useMaximalCoordinates = True
# half side length of each box segment of the snake
halfBoxLen = 0.25
# create a box
# when you create something, an index is returned that can be used to refer to the object
colBoxId = p.createCollisionShape(p.GEOM_BOX, halfExtents=[halfBoxLen, halfBoxLen, halfBoxLen])

link_Masses = [] # mass of each segment
linkCollisionShapeIndices = [] # id of collision shape for each segment
linkVisualShapeIndices = [] # id of visual object for each segment
linkPositions = [] # position of links (joint) to connect segments
linkOrientations = [] # orientation of links (joint) to connect segments
linkInertialFramePositions = [] # link inertial frame positions for each join
linkInertialFrameOrientations = [] # link inertial frame orientation for each joint
indices = [] # indices of each component of the snake
jointTypes = [] # types of each link
axis = [] # axis of each link

# create 6 more segments
# in the next part, the main segment is created so this makes a total of 7 segments
for i in range(6):
    link_Masses.append(1) # mass = 1
    linkCollisionShapeIndices.append(colBoxId) # each segment uses same collision template
    linkVisualShapeIndices.append(-1) # no visual object added (use default coloring)
    linkPositions.append([0, halfBoxLen * 2.0 + 0.01, 0]) # offset each next link relative to the previous
    linkOrientations.append([0, 0, 0, 1]) # set link orientations
    linkInertialFramePositions.append([0, 0, 0]) # set link inertial frame positions
    linkInertialFrameOrientations.append([0, 0, 0, 1]) # set link inertial frame orientations
    indices.append(i) # add indices of components
    jointTypes.append(p.JOINT_REVOLUTE) # all joints are of type JOINT_REVOLUTE
    axis.append([0, 0, 1]) # each joint revolves around z axis

# head initial position and orientation
basePosition = [(11.5 + WORLD_OFFSET_X) * SCALE, (9 + WORLD_OFFSET_Y) * SCALE, 1]
baseOrientation = [0, 0, 0, 1]
snakeId = p.createMultiBody(1,
                              colBoxId,
                              -1,
                              basePosition,
                              baseOrientation,
                              linkMasses=link_Masses,
                              linkCollisionShapeIndices=linkCollisionShapeIndices,
                              linkVisualShapeIndices=linkVisualShapeIndices,
                              linkPositions=linkPositions,
                              linkOrientations=linkOrientations,
                              linkInertialFramePositions=linkInertialFramePositions,
                              linkInertialFrameOrientations=linkInertialFrameOrientations,
                              linkParentIndices=indices,
                              linkJointTypes=jointTypes,
                              linkJointAxis=axis,
                              useMaximalCoordinates=useMaximalCoordinates)

# set gravity down on the z axis
p.setGravity(0, 0, -10)
p.setRealTimeSimulation(0)

# set friction and joint dynamics
anistropicFriction = [1, 0.01, 0.01]
p.changeDynamics(snakeId, -1, lateralFriction=2, anisotropicFriction=anistropicFriction)
p.getNumJoints(snakeId)
for i in range(p.getNumJoints(snakeId)):
    p.getJointInfo(snakeId, i)
    p.changeDynamics(snakeId, i, lateralFriction=2, anisotropicFriction=anistropicFriction)

# create and display all obstacles
for obs in OBSTACLES:
    createObstacle(obs[0][0], obs[0][1], obs[1][0], obs[1][1])
    
dt = 1. / 240.
SNAKE_NORMAL_PERIOD = 0.1
m_wavePeriod = SNAKE_NORMAL_PERIOD

STEERING_CONST = 0.2

# snake constants
m_waveLength = 4
m_wavePeriod = 1.5
m_waveAmplitude = 0.4
m_waveFront = 0.0
#our steering value
m_steering = 0.0
m_segmentLength = halfBoxLen * 2.0

# read in waypoints that were generated
file = open("waypoints_50.data", "rb")
waypoints = np.load(file)
file.close()
waypoints[:, 0] += WORLD_OFFSET_X
waypoints[:, 1] += WORLD_OFFSET_Y
waypoints *= SCALE
waypoint_index = 0

# display waypoints
for w in waypoints:
    wid = p.createVisualShape(
        p.GEOM_SPHERE,
        radius=TOLERANCE,
        rgbaColor=[0, 0, 0, 1],
    )
    wmb = p.createMultiBody(
        0,
        -1,
        wid,
        [w[0], w[1], 0.25]
    )

# keep going until the final waypoint has been reached
while (waypoint_index < len(waypoints)):
    base_state = p.getBasePositionAndOrientation(snakeId)
    robot_pos = base_state[0]
    robot_orien = p.getEulerFromQuaternion(base_state[1])
    robot_angle = robot_orien[2]
    robot_angle -= math.pi / 2
    robot_angle = fix_angle(robot_angle)

    goal_pos = waypoints[waypoint_index]

    if distance(robot_pos, goal_pos) < TOLERANCE:
        waypoint_index += 1
    
    ray = (goal_pos[0] - robot_pos[0], goal_pos[1] - robot_pos[1])
    goal_angle = math.atan2(ray[1], ray[0])
    goal_angle = fix_angle(goal_angle)

    rel_angle = goal_angle - robot_angle
    rel_angle = fix_angle(rel_angle)

    if rel_angle > 0:
        m_steering = -STEERING_CONST
    else:
        m_steering = STEERING_CONST

    numMuscles = p.getNumJoints(snakeId)
    scaleStart = 1.0

    #start of the snake with smaller waves.
    if (m_waveFront < m_segmentLength * 4.0):
        scaleStart = m_waveFront / (m_segmentLength * 4.0)

    segment = numMuscles - 1

    #we simply move a sin wave down the body of the snake.c
    for joint in range(p.getNumJoints(snakeId)):
        segment = joint
        #map segment to phase
        phase = (m_waveFront - (segment + 1) * m_segmentLength) / m_waveLength
        phase -= math.floor(phase)
        phase *= math.pi * 2.0

        #map phase to curvature
        targetPos = math.sin(phase) * scaleStart * m_waveAmplitude

        # steer snake by squashing +ve or -ve side of sin curve
        if (m_steering > 0 and targetPos < 0):
            targetPos *= 1.0 / (1.0 + m_steering)

        if (m_steering < 0 and targetPos > 0):
            targetPos *= 1.0 / (1.0 - m_steering)

        #set our motor
        p.setJointMotorControl2(
            snakeId,
            joint,
            p.POSITION_CONTROL,
            targetPosition=targetPos + m_steering,
            force=30
        )

    #wave keeps track of where the wave is in time
    m_waveFront += dt / m_wavePeriod * m_waveLength
    p.stepSimulation()

    time.sleep(dt)

# Results
As you can see, the planning algorithm was successful in finding a proper path to get from the start to the end goal in a moderately efficient manner.  The idea for this is not to necessarily be the optimal path, but it should be a somewhat reasonable path.  The constraints on the motion provide a proper path for the snake robot.  Although the snake can turn somewhat sharply, it cannot necessarily turn immediately when it wants to.  It moves by following an S-curve path on the ground.  So as it is curving towards the left, but it wants to turn to the right, it won't be able to accomplish this immediately.
Originally, I wanted to implement this problem using RRT* as it is usually faster than RRT in terms of both search time and path distance.  However, when I started to add in the constraints on motion and modeling using them, I quickly realized that RRT* would no longer be possible due to RRT*'s update of shorter path.  Because of this, I decided to stick with RRT.
The generated path works well, it obeys all the obstacle and margin avoidance it is supposed to and is still able to create a possible path from the start to the finish.  The simulation works well and proves that the generated path and implemented controller work.  If it didn't work, then the snake would not follow the path or would get stuck somewhere.
A video of the robot moving through the maze using the planner can be viewed through the link below.
https://www.youtube.com/watch?v=Z-EIw-ufno8

# Further Work
No matter what project it is, there are always opportunities for future work or exploration in a similar area.  For this project, some future extensions could be to test in different and more complex environments, and the ability to handle dynamic environments.
More complex environments could be extending this project to a fully 3D planning environment which would have different verticalities and different terrain.  The different terrain could be things like rocks and rubble that the robot has to decide if it wants to move along by traversing them or going around it.
A dynamic environment is one in which there are objects in it that are not static.  In all of my testing, I used static obstacles that did not move over time.  In a dynamic environment, obstacles that were in one place at one time may be in a completely different place later on.  This adds difficulty because the robot must be constantly updating its representation of the world as well as predicting where the dynamic object might be in the future to make sure that it isn't going to collide with it during its motionn.